In [37]:
import pandas as pd
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from itertools import chain

In [2]:
# Load the saved XGBoost model
loaded_model = xgb.Booster()
model_file_path = "/Users/halasadi/code/pmhc_methods_tf/internal_data/2023_12_14_9mer_xgboost_model.json"
loaded_model.load_model(model_file_path)

In [29]:
# load in the human proteome from here: https://www.uniprot.org/proteomes/UP000005640

fasta_file_path = "/Users/halasadi/code/pmhc_methods_tf/external_data/mixmhcpred/UP000005640_9606.fasta"
sequences = SeqIO.parse(fasta_file_path, "fasta")
peptides = list()
cnt = 1
for record in sequences:
    cnt += 1
    peptide = str(record.seq)
    if (len(peptide) >= 8):
        peptides.append(peptide)

In [33]:
def generate_all_kmers(input_string, k):
    kmers = []
    # Iterate through the input string to generate 9-mers
    for i in range(len(input_string) - k + 1):
        kmer = input_string[i:i + k]
        if 'N' not in kmer:
            kmers.append(kmer)
    return(kmers)
    

In [35]:
all_kmers = list()
for peptide in peptides:
    all_kmers.append(generate_all_kmers(peptide, 9))
all_kmers = list(chain.from_iterable(all_kmers))

In [41]:
import random
sampled_kmers = random.sample(all_kmers, 10000)

In [42]:
from sklearn.preprocessing import OneHotEncoder

In [43]:
encoder = OneHotEncoder()

In [44]:
X_encoded = np.array([list(s) for s in sampled_kmers])
X_encoded = encoder.fit_transform(X_encoded).toarray()

In [45]:
X_encoded[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0.])

In [55]:
ypred = loaded_model.predict(X_encoded)

TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'numpy.ndarray'>)